# Sistema de Recomendação utilizando SVD

##  Introdução

A evolução da internet trouxe novas tecnologias como a de streaming de filmes e músicas nas quais os usuários têm acesso a milhares de produtos em um catálogo online sem ter a necessidade de baixa-los. O comercio eletrônico também sofre uma grande evolução nos últimos anos, além de grande número de sites oferecendo uma vasta gama de produtos tem-se plataformas de comparação de preços e sugestões de produtos para os usuários.


Com essa evolução toda o usuário tem disponível uma vasta quantidade de produtos a comprar nos marketplaces, ou filmes e músicas para assistir e ouvir, mas qual produto vai de acordo com o que o usuário precisa/quer comprar naquele momento? Ou qual filme ou música está mais adequado ao seu estilo? Neste sentido, surge a necessidade de uma recolha e organização da informação a ser oferecida ao usuário final para que este compre ou consuma o serviço que de fato necessita. Desta necessidade surgiram os sistemas de recomendação ou Recommender Systems (RS).

Os sistemas de recomendação possuem diferentes técnicas que diferem entre si pela forma
como cada uma reúne e trata a informação relativa às preferências dos usuários. Dentro dos sistemas de recomendação um dos mais utilizados é a filtragem colaborativa ou collaborative filtering systems (CFS) que têm por base o seguinte pressuposto: se dois utilizadores classificaram/adquiriram produtos semelhantes no passado, então estes irão classificar/adquirir produtos semelhantes no futuro.


Em 2006 a Netflix abre um concurso que premia com US$1 milhão quem criar um sistema de recomendação mais eficiente e em 2009 a equipe vencedora implementando um algoritmo de filtragem colaborativa foi capaz de criar um sistema que oferece um erro 10% inferior ao do sistema utilizado pelo próprio Netflix. Em 2016 a Netflix mudou novamente o seu sistema de recomendação, criou-se um novo algoritmo que separa os assinantes em comunidades globais, independentemente de sua localização, e leva em consideração os gostos e preferências pessoais de cada usuário.

## Tipos de Sistemas de Recomendação

- Sistemas baseadas em conteúdo ou content-based systems (CBS). Utilizam a informação sobre o conteúdo dos itens vistos no passado para recomendar novos itens.



- Sistemas baseadas em filtragem colaborativa ou collaborative filtering systems (CFS). É utilizada a informação da matriz utilizador-item.



- Sistemas baseadas em filtragem híbrida ou hybrid recommender systems (HRS). Estes sistemas combinam essencialmente técnicas baseadas em filtragem colaborativa e em conteúdo com o objetivo de mitigar as falhas apresentadas por cada método quando implementado individualmente.


## Filtragem colaborativa ou collaborative filtering systems (CFS)

Neste trabalho vamos utilizar o dataset MovieLens 20M que contem 20.000.263 ratings (avaliações) realizadas por 138.493 usuários aplicadas a 27.278 filmes. Como esse dataset apresenta as notas dadas pelos usuários aos filmes optou-se por utilizar o sistema de recomendação por filtragem colaborativa.
Esse sistema faz recomendações de itens desconhecidos a um usuário com base nos itens classificados anteriormente por outros cujo perfil ou gostos sejam similares ao do usuário ativo.

As técnicas de filtragem colaborativa, por sua vez, podem ser estratificadas em duas outras classes distintas:

- Baseado em memória (memory-based ou neighborhoood-based): recomendações computadas com base na matriz user-item, completa ou amostral, de ratings; a representação da matriz é mantida em memória. 


- Baseado em modelo (model-based ou latent factor model): utilizam ratings dos usuários para construir um “modelo” capaz de fazer predições. Esses modelos são capazes de representar características ocultas de usuários e itens, normalmente construídos via técnica de aprendizagem de máquina.


## Filtragem colaborativa Baseada em Modelo

Neste trabalho, utilizando o dataset MovieLens as métricas de medições se mostraram mais eficazes utilizando a Filtragem colaborativa Baseada em Modelo. Desta forma o tratamento deste dataset é demonstrado utilizando esse tipo de sistema de recomendação com a aplicação da técnica de decomposição de matrizes chamada de Singular Value Decomposition (SVD).

## Singular Value Decomposition (SVD)

SVD é uma técnica de decomposição de matrizes que tem como consequência a redução da dimensionalidade de um dataset. Em suma ela tenta escolher as melhores características dos dados da matriz completa e a decompõe numa matriz de menor dimensão com as características que mais se destacam. Maiores detalhes técnicos serão apresentados mais adiante na aplicação do algoritmo.

## Início do projeto

### Importação de dados MovieLens 20M

In [1]:
# importando bibliotecas principais
import numpy as np
import pandas as pd

In [2]:
#importando dados
ratings = pd.read_csv('ratings.csv')
movies=pd.read_csv('movies.csv')

In [3]:
#análise rápida dos dados:
ratings.head()


,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


### Tratamento dos dados

Nesta etapa deve-se tratar os dados para obter a matriz que contenha em sua coluna os IDs dos filmes, nas linhas os IDs dos usuários e os elementos do interior como sendo as notas (ratings). Vamos começar eliminando a coluna 'timestamp' do dataframe ratings:

In [5]:
ratings2=ratings.drop(columns='timestamp')

Resultando:

In [6]:
ratings2.head()

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5


Vamos agora verificar o tamanho do dataframe:

In [7]:
#verificando o comprimento
len(ratings2)

20000263

Como este dataframe é muito grande e o meu computador não tem lá essas coisas de memória optou-se por seccionar a matriz e tratar menos dados:

In [8]:
ratings2 = ratings2.iloc[:1000000,:]

In [9]:
#verificando o comprimento
len(ratings2)

1000000

Nesta etapa ja podemos modificar o formato do dataframe "ratings2" para o formato de matriz no qual as linhas serão os IDs dos usuários, as colunas os IDs dos filmes e os elementos as notas dadas:

In [10]:
pivot_table = ratings2.pivot_table(index = 'userId',columns = 'movieId',values = 'rating')
pivot_table.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,129350,129354,129428,129707,130052,130073,130219,130462,130490,130642
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Pela análise da tabela acima tem-se que vários elementos do dataframe foram preenchidos por "NaN". Isso ocorre pelo fato de que os usuários não assistem e nem classificam todos os filmes apenas os que já assistiram. São esses filmes NÃO classificados que o sistema de recomendação deve atuar sugerindo de maneira "artificial" notas que o usuário daria para os filmes.

Porém, para que o algoritmo trabalhe no sistema de recomendação a matriz a ser tratada deve conter números racionais e necessita-se retirar o termo "NaN" do dataframe. Para isso optou-se por preencher os elementos de "NaN" por zeros:

In [11]:
df = pivot_table.fillna(0)
df.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,129350,129354,129428,129707,130052,130073,130219,130462,130490,130642
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Próximo passo é modificar o tipo de "df" de dataframe para matriz(numpy array):

In [12]:
matriz=df.as_matrix()
matriz

C:\Users\Bruno\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([[0. , 3.5, 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 4. , ..., 0. , 0. , 0. ],
       [4. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [4. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [4. , 3. , 3. , ..., 0. , 0. , 0. ]])

Normalização da matriz pela média:

In [15]:
#calculo da média por usuário
usuarios_media = np.mean(matriz, axis = 1)
#normalização pela média
matriz_normalizada = matriz - usuarios_media.reshape(-1, 1)

In [16]:
matriz_normalizada

array([[-0.04695341,  3.45304659, -0.04695341, ..., -0.04695341,
        -0.04695341, -0.04695341],
       [-0.01749104, -0.01749104,  3.98250896, ..., -0.01749104,
        -0.01749104, -0.01749104],
       [ 3.94473118, -0.05526882, -0.05526882, ..., -0.05526882,
        -0.05526882, -0.05526882],
       ...,
       [ 3.98089606, -0.01910394, -0.01910394, ..., -0.01910394,
        -0.01910394, -0.01910394],
       [-0.00637993, -0.00637993, -0.00637993, ..., -0.00637993,
        -0.00637993, -0.00637993],
       [ 3.95189964,  2.95189964,  2.95189964, ..., -0.04810036,
        -0.04810036, -0.04810036]])

Com a matriz no formato correta e normalizada pode-se aplicar o método de Descomposição do valor singular (SVD)

### Support Vector Decomposition (SVD)

Nesta etapa aplica-se o método de fatoração da matriz utilizando a decomposição do valor singular (Singular value decomposition - SVD). 

O método SVD, é uma técnica de fatoração de matrizes que decompõe uma matriz A de dimensão m × n como um produto de três matrizes, dado pela equação abaixo:

$A$ = $U$.$S$.$V^{T}$

A matriz  $U$ representa o quanto que os usuários "gostam" de uma determinada característica, a matriz $V$ representa o quanto cada característica é relevante para os filmes. A matriz $S$ é a matriz diagonal dos valores singulares. Para ter uma boa precisão trunca-se as matrizes nas melhores "k" características

As bibliotecas Scipy e Numpy são muito utilizadas para a decomposição SVD. Neste trabalho optou-se por utilizar a biblioteca Scipy: 

In [17]:
from scipy.sparse.linalg import svds
U, S, Vt = svds(matriz_normalizada, k = 20)

Tornando $S$ uma matriz diagonal:

In [18]:
s = np.diag(S)

Com as matrizes U, S e V encontradas podemos multiplica-las, conforma a equação acima, para encontrar a matriz A que é a matriz com as previsões de notas dos usuários. 

In [19]:
notas_previsoes=np.dot(np.dot(U, s),Vt)

Vamos agora voltar a "média" das notas retiradas na normalização anterior:

In [20]:
previsoes = notas_previsoes + usuarios_media.reshape(-1, 1)

E tornar a matriz em um dataframe:

In [21]:
previsao_final=pd.DataFrame(previsoes, columns=df.columns)

Logo a matriz de previsões com todas as possíveis classificações dos usuários para os filmes é dada por:

In [23]:
previsao_final.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,129350,129354,129428,129707,130052,130073,130219,130462,130490,130642
0,1.179880,0.558732,-0.256543,0.051029,-0.455241,0.228345,-0.665528,0.007794,-0.119109,-0.198906,...,0.009026,-0.002205,-0.014022,0.004354,0.006061,0.005113,-0.002541,0.005481,0.001544,0.009572
1,0.890696,0.199455,0.200046,-0.003052,0.162159,0.188276,0.289070,0.005551,0.098461,0.037543,...,0.002046,0.004582,-0.000494,0.002057,0.002095,0.007814,0.004238,0.003549,0.006766,0.002530
2,1.910463,0.438245,0.070561,-0.096553,-0.125537,0.172137,0.036586,-0.019875,-0.024364,0.041606,...,-0.002802,0.001560,-0.021835,-0.006250,-0.006312,0.009672,-0.007165,-0.000191,0.002044,-0.005844
3,0.397016,0.716549,0.329797,0.076312,0.329305,0.497436,0.274108,0.071601,0.154943,1.103551,...,0.000055,-0.000341,0.001345,0.000432,0.000730,-0.000710,0.002064,0.001218,-0.001211,0.001438
4,3.392020,1.246280,1.141989,0.128417,1.069150,1.295842,1.278474,0.133435,0.362326,1.514370,...,0.004481,0.004852,0.007907,0.003950,0.003722,0.007205,0.003504,0.005351,0.008008,0.003498


Foi elaborado a função "recomendar" com dois objetivos. A primeira é retornar os "n" primeiros filmes mais bem avaliados originalmente pelo usuário, a esta função de retorno deu-se o nome de "usuario_notas". O segundo é retornar os "n" primeiros filmes mais recomendados para o usuário assistir, a este retorno deu-se o nome de "recommendacoes" conforme mostra a seguir:

In [29]:
def recomendar(preds, userID, movies, ratings, n_recommendacoes):
    
    # Para começar em 0 pois no dataframe df userID começava em 1
    numero_usuario = userID - 1 
      
    # Monta um dataframe somente com os filmes classificados pelo usuário escolhido.
    df_usuario = ratings[ratings['userId']==userID]
    usuario_notas = (df_usuario.merge(movies, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False)   
                )

   # seleciona na nova matrix a linha do usuário e classifica em ordem decrescente seus filmes
    predicoes_usuarios = previsao_final.iloc[numero_usuario].sort_values(ascending=False)
    
    # Monta um dataframe com os nomes e generos dos filmes que o usuário ainda não deu nota.
    recommendacoes = (movies[~movies['movieId'].isin(usuario_notas['movieId'])].
         merge(pd.DataFrame(predicoes_usuarios).reset_index(), how = 'left',
               left_on = 'movieId',
               right_on = 'movieId').
         rename(columns = {numero_usuario: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:n_recommendacoes, :-1]
                      )
   

    return usuario_notas, recommendacoes

Para aplicar a a função basta escolher o numero do usuário (userID), e o número de recomendações(n_recommendacoes) conforme mostra a expressão a seguir: 

In [46]:
usuario_notas, recommendacoes = recomendar(previsao_final, 4 ,movies, ratings,  5)

Os 5 primeiros filmes mais bem avaliados originalmente:

In [49]:
usuario_notas.head()

,userId,movieId,rating,timestamp,title,genres
27,4,733,5.0,840879322,"Rock, The (1996)",Action|Adventure|Thriller
16,4,454,5.0,840878944,"Firm, The (1993)",Drama|Thriller
11,4,377,4.0,840878994,Speed (1994),Action|Romance|Thriller
26,4,596,4.0,840879424,Pinocchio (1940),Animation|Children|Fantasy|Musical
25,4,594,4.0,840879265,Snow White and the Seven Dwarfs (1937),Animation|Children|Drama|Fantasy|Musical


Os 5 primeiros mais mais recomendados pelo sistema para o usuário 4:

In [48]:
recommendacoes

,movieId,title,genres
436,457,"Fugitive, The (1993)",Thriller
477,500,Mrs. Doubtfire (1993),Comedy|Drama
561,592,Batman (1989),Action|Crime|Thriller
564,597,Pretty Woman (1990),Comedy|Romance
144,150,Apollo 13 (1995),Adventure|Drama|IMAX


### Avaliação do algoritmo

Existem muitas métricas de avaliação, mas optou-se pela métrica da Raiz da média dos erros quadrados (Root Mean Squared Error - RMSE), por ser uma das mais utilizadas que inclusive foi utilizada no campeonato de sistemas de recomendação proposto pela Netflix. A formula é dada a seguir:

<img src="https://latex.codecogs.com/gif.latex?RMSE&space;=\sqrt{\frac{1}{N}&space;\sum&space;(x_i&space;-\hat{x_i})^2}" title="RMSE =\sqrt{\frac{1}{N} \sum (x_i -\hat{x_i})^2}" />

Para executar esta métrica utilizou-se a biblioteca "Surprise" que apresenta o algoritmo de medição do RMSE para SVD.

In [51]:
# Importar as bibliotecas
from surprise import Reader, Dataset, SVD, evaluate

# Lendo a Biblioteca Reader
reader = Reader()

# Carregando o conjunto de dados
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Quebrando o dataset em varios blocos(n_folds)
data.split(n_folds=2)

In [52]:
# Aplicando o algoritmo SVD.
svd = SVD()

# Calculando RMSE do algoritmo SVD.
evaluate(svd, data, measures=['RMSE'])

C:\Users\Bruno\Anaconda3\lib\site-packages\surprise\evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
C:\Users\Bruno\Anaconda3\lib\site-packages\surprise\dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE of algorithm SVD.

------------
Fold 1
RMSE: 0.8122
------------
Fold 2
RMSE: 0.8123
------------
------------
Mean RMSE: 0.8123
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.81220482266577, 0.8123274305197772]})

### Conclusão

Como a medição encontrada foi de 81,23% tem-se que a aplicação do algoritmo SVD é uma técnica muito eficiênte para ser utilizada em sistemas de recomendções quando se conhece as avaliações de produtos feitas pelos usuários.